In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel('Superstore.xls')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [3]:
df.shape

(9994, 21)

In [4]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [5]:
## Recency
rec = df.groupby('Customer ID')['Order Date'].max().reset_index()
rec.head()

,Customer ID,Order Date
0,AA-10315,2017-06-29
1,AA-10375,2017-12-11
2,AA-10480,2017-04-15
3,AA-10645,2017-11-05
4,AB-10015,2016-11-10


In [6]:
print(df['Order Date'].min())
print(df['Order Date'].max())

2014-01-03 00:00:00
2017-12-30 00:00:00


In [7]:
rec['Recency'] = (df['Order Date'].max() - rec['Order Date']).dt.days
rec.head()

,Customer ID,Order Date,Recency
0,AA-10315,2017-06-29,184
1,AA-10375,2017-12-11,19
2,AA-10480,2017-04-15,259
3,AA-10645,2017-11-05,55
4,AB-10015,2016-11-10,415


In [8]:
## Frequency
freq = df['Customer ID'].value_counts().reset_index()
freq.columns = ['Customer ID','Frequency']
freq.head()

,Customer ID,Frequency
0,WB-21850,37
1,JL-15835,34
2,PP-18955,34
3,MA-17560,34
4,SV-20365,32


In [9]:
print(rec.shape)
print(freq.shape)

(793, 3)
(793, 2)


In [10]:
## Monetory
mon = df.groupby(['Customer ID'])['Sales'].sum().reset_index()
mon.columns = ['Customer ID', 'Monetory']
mon.head()

,Customer ID,Monetory
0,AA-10315,5563.560
1,AA-10375,1056.390
2,AA-10480,1790.512
3,AA-10645,5086.935
4,AB-10015,886.156


In [11]:
mon.shape

(793, 2)

In [12]:
rf = pd.merge(rec,freq, on='Customer ID')
rf.head()

,Customer ID,Order Date,Recency,Frequency
0,AA-10315,2017-06-29,184,11
1,AA-10375,2017-12-11,19,15
2,AA-10480,2017-04-15,259,12
3,AA-10645,2017-11-05,55,18
4,AB-10015,2016-11-10,415,6


In [13]:
rfm = pd.merge(rf,mon, on='Customer ID')
rfm.head()

,Customer ID,Order Date,Recency,Frequency,Monetory
0,AA-10315,2017-06-29,184,11,5563.560
1,AA-10375,2017-12-11,19,15,1056.390
2,AA-10480,2017-04-15,259,12,1790.512
3,AA-10645,2017-11-05,55,18,5086.935
4,AB-10015,2016-11-10,415,6,886.156


In [14]:
rfm.drop('Order Date',axis=1,inplace=True)
rfm.head()

,Customer ID,Recency,Frequency,Monetory
0,AA-10315,184,11,5563.560
1,AA-10375,19,15,1056.390
2,AA-10480,259,12,1790.512
3,AA-10645,55,18,5086.935
4,AB-10015,415,6,886.156


In [15]:
rfm1 = rfm.copy()
rfm2 = rfm.copy()

In [16]:
## Quartile analysis, decile analysis and pentile analysis
## Pentile analysis
z1 = rfm.describe(percentiles=[0.20,0.40,0.60,0.80]).T
z1 = z1.iloc[:,3:]
z1

,min,20%,40%,50%,60%,80%,max
Recency,0.000,26.0000,53.0000,75.000,103.000,224.6000,1165.00
Frequency,1.000,7.0000,10.0000,12.000,13.000,18.0000,37.00
Monetory,4.833,961.5406,1739.4324,2256.394,2750.748,4282.6344,25043.05


In [30]:
rfm['RScore'] = pd.cut(rfm['Recency'],bins=[-1,26,53,103,224.6,1165],labels=[5,4,3,2,1])
rfm['FScore'] = pd.cut(rfm['Frequency'],bins=[0,7,10,13,18,37],labels=[1,2,3,4,5])
rfm['MScore'] = pd.cut(rfm['Monetory'],bins=[4.83, 961.5406, 1739.4324, 2750.748, 4282.6344, 25043.05],labels=[1,2,3,4,5])
rfm.head()

,Customer ID,Recency,Frequency,Monetory,RScore,FScore,MScore,RF_Score,RFM_Score,Segment
0,AA-10315,184,11,5563.560,2,3,5,23,235,At Risk
1,AA-10375,19,15,1056.390,5,4,2,54,542,Champions
2,AA-10480,259,12,1790.512,1,3,3,13,133,At Risk
3,AA-10645,55,18,5086.935,3,4,5,34,345,Loyal Customers
4,AB-10015,415,6,886.156,1,1,1,11,111,Hibernating


In [31]:
rfm['RF_Score'] = rfm['RScore'].astype(str) + rfm['FScore'].astype(str)
rfm['RFM_Score'] = rfm['RScore'].astype(str) + rfm['FScore'].astype(str) + rfm['MScore'].astype(str)
rfm.head()

,Customer ID,Recency,Frequency,Monetory,RScore,FScore,MScore,RF_Score,RFM_Score,Segment
0,AA-10315,184,11,5563.560,2,3,5,23,235,At Risk
1,AA-10375,19,15,1056.390,5,4,2,54,542,Champions
2,AA-10480,259,12,1790.512,1,3,3,13,133,At Risk
3,AA-10645,55,18,5086.935,3,4,5,34,345,Loyal Customers
4,AB-10015,415,6,886.156,1,1,1,11,111,Hibernating


In [32]:
rfm['Segment'] = rfm['RF_Score'].replace({
    '[1-2][1-2]': 'Hibernating',
    '3[1-2]' : 'About to Sleep',
    '41': 'Promising',
    '51': 'New Customers',
    '33': 'Needs Attention',
    '[1-2][3-4]': 'At Risk',
    '[1-2]5': 'Cant Loose them',
    '[4-5][2-3]': 'Potential Customers',
    '[3-4][4-5]': 'Loyal Customers',
    '5[4-5]': 'Champions'
},regex=True)

In [33]:
rfm['Segment'].value_counts()

Segment
Hibernating            168
Loyal Customers        137
Potential Customers    119
At Risk                113
Champions               85
About to Sleep          67
Cant Loose them         35
Needs Attention         29
New Customers           22
Promising               18
Name: count, dtype: int64

In [35]:
profile = rfm.groupby(['Segment']).agg({'Segment':'count',
                              'Recency':'mean','Frequency':'mean','Monetory':'mean'})
profile

,Segment,Recency,Frequency,Monetory
Segment,,,,
About to Sleep,67,74.238806,7.462687,1960.725428
At Risk,113,251.000000,13.610619,3176.060832
Cant Loose them,35,232.257143,22.800000,5862.524929
Champions,85,13.552941,18.858824,3959.885987
Hibernating,168,355.964286,6.583333,1300.241243
Loyal Customers,137,55.708029,19.065693,4501.911668
Needs Attention,29,78.724138,12.000000,2648.245566
New Customers,22,12.727273,5.681818,1000.484218
Potential Customers,119,26.025210,10.655462,2616.982713
